# Continuous Control

Run this notebook end-to-end to complete Project 2 - Continuous Control of Udacity's Deep Reinforcement Learning Nanodegree program.

---

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [2]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.30 which is incompatible.
jupyter-console 6.4.3 has requirement jupyter-client>=7.0.0, but you'll have jupyter-client 5.2.4 which is incompatible.


Import all python packages needed to execute the notebook.

In [4]:
#!curl -O https://raw.githubusercontent.com/udacity/workspaces-student-support/master/jupyter/workspace_utils.py

In [5]:
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import torch
from unityagents import UnityEnvironment
from workspace_utils import active_session

The environment is already saved in the Workspace and can be accessed at the file path provided below. However, if running this notebook locally, or using the multi-arm environment, change `PATH_TO_ENV` to the appropriate file path.

(Note: The Unity ML environments are saved in the git repo for this project)

In [6]:
# PATH_TO_ENV = "/data/Reacher_Linux_NoVis/Reacher.x86_64". # multi-agent path
PATH_TO_ENV = "/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64"  # single agent path
env = UnityEnvironment(file_name=PATH_TO_ENV)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


**Aside**: Unity environments contain <b>*brains*</b> which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [7]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [8]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print(f'Number of agents: {num_agents}')

# size of each action
ACTION_SIZE = brain.vector_action_space_size
print(f'Size of each action: {ACTION_SIZE}')

# examine the state space 
states = env_info.vector_observations
N_AGENTS = states.shape[0]
STATE_SIZE = states.shape[1]
print(f'There are {N_AGENTS} agents. Each observes a state with length: {STATE_SIZE}')
print(f'The state for the first agent looks like: {states[0]}')

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726671e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

In [9]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(N_AGENTS)                            # initialize the score (for each agent)
while True:
    actions = np.random.randn(N_AGENTS, ACTION_SIZE)   # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.0


### 4. Train the agent

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:

- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:

```python
env_info = env.reset(train_mode=True)[brain_name]
```

- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

- All of the source code (`Agent`, `Model`, `ReplayBuffer`) are defined under `src/`. Feel free to inspect these files to see explicitly what each is contributing to the final result.

The algorithm being used is the **Deep Deterministic Policy Gradient (DDPG)**. The algorithm learns both a policy and a value function at the same time through the use of an Actor (the policy) and a Critic (the value function estimator). DDPG is often considered Deep Q-Learning but when the action space is continuous, as it is for the environment we are trying to solve.

More information about the different components of DDPG are below:

* Model:

* Agent:

* Replay Buffer:

The DDPG training algorithm is defined in the function below.

In [10]:
def ddpg(agent,
         brain_name,
         n_episodes=1000,
         max_t=1000,
         print_every=10):
    """Deep Deterministic Policy Gradient
    
    Params
    ======
        agent (Agent): the agent to use for training
        brain_name (str): the brain name to use for the environment
        n_episodes (int): maximum number of training episodes
        max_t (int): how long to let an individual training episode run
        print_every (int): how frequently to log
    """
    
    scores = []                        # list continaing scores for each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    max_score = -np.Inf                # best score observed
    
    for i_episode in range(1, n_episodes+1):
        
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        state = env_info.vector_observations[0]            # get the current state
        agent.reset()                                      # reset the agent
        score = 0                                          # initilize the episode score
        t0 = time.time()
        for t in range(max_t):
            action = agent.act(state)                      # select an action
            
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            
            agent.step(state, action, reward, next_state, done, t)
            
            score += reward                                # update the score
            state = next_state                             # roll over the state to next time step
            if done:
                break 
        
        elapsed_time = time.time() - t0
        scores_window.append(score)  # save the most recent score
        scores.append(score)         # save the most recent score
        
        if i_episode % print_every == 0:
            print('\rEpisode {} ({}s)\tAverage Score: {:.2f}'.format(i_episode, print_every * round(elapsed_time), np.mean(scores_window)))
        
        if np.mean(scores_window)>=13.0:
            print('\n\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            best_actor = agent.actor_local.state_dict()
            best_critic = agent.critic_local.state_dict()
            break  
    
    return scores, best_actor, best_critic

Import the agent and train DDPG.

In [11]:
from src.ddpg import Agent

agent = Agent(state_size=STATE_SIZE, action_size=ACTION_SIZE, random_seed=0)

Keep the environment running with helper function

In [12]:
t0 = time.time()

with active_session():
    scores, best_actor, best_critic = ddpg(agent, brain_name)

t1 = time.time()
elapsed_time = (t1 - t0) / 60.0
print("\nElapsed Time: {0:3.2f} mins.".format(elapsed_time))

/home/workspace/drlnd-p2-continuous-control/src/ddpg.py:113: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), GRAD_CLIPPING)


Episode 10 (120s)	Average Score: 0.75
Episode 20 (120s)	Average Score: 0.83
Episode 30 (120s)	Average Score: 0.84
Episode 40 (120s)	Average Score: 0.87
Episode 50 (120s)	Average Score: 0.92
Episode 60 (120s)	Average Score: 0.91
Episode 80 (120s)	Average Score: 0.92
Episode 90 (120s)	Average Score: 0.91
Episode 100 (120s)	Average Score: 0.89
Episode 110 (120s)	Average Score: 0.90
Episode 120 (120s)	Average Score: 0.88
Episode 130 (120s)	Average Score: 0.80
Episode 140 (120s)	Average Score: 0.71


KeyboardInterrupt: 

When finished training, you can close the environment.

In [ ]:
env.close()

### 5. Visualizing results and saving best model



In [ ]:
# plot the scores
fig = plt.figure(figsize=(9, 6))
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores, label="Raw Score")

# plot the moving average
scores_series = pd.Series(scores)
windows = scores_series.rolling(100)
ma = windows.mean().dropna()
x, y = ma.index, ma.values
plt.plot(x, y, label="Moving Average")

plt.xlabel("Episode #")
plt.ylabel("Score")
plt.title("Agent performance over time")
plt.legend()

fig.savefig("images/performance.png")

In [ ]:
# save model
torch.save(best_actor, "actor_checkpoint.pth")
torch.save(best_critic, "critic_checkpoint.pth")

### 6. Potential Improvements

Some additional ideas that could lead to better performance or interesting insights include:

* Different model architectures (more/less layers, larger/smaller layer sizes)
* Prioritized replay buffer
* Training on the 20-arm environment versus the 1-arm environment
* Different policy gradient algorithms like A3C or SAC